In [1]:
import time
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import numpy as np
import pandas as pd

<h1># 01 - Ergebnisdaten: Parsen der Ergebnisdaten - Tokyo (2016 - 2019)</h1>
<hr>
<p><b>Hinweis:</b> Dateifpade sind absolut angegeben und müssen entsprechend der eigenen Verzeichnisstruktur angepasst werden!</p>
<hr>

In [2]:
# Hilfsfunktion: Encoding/Decoding für einige Felder notwendig, da sonst das Parsen nicht funktioniert
def get_decoded_element(content, index):
    c_latin = str(content).encode('latin-1','replace')
    c_utf8 = c_latin.decode('utf8')
    c_utf8_length = len(c_utf8.split("?"))
    return c_utf8.split("?")[c_utf8_length-index]

In [3]:
for year in range(2016, 2020):
    # Treiber laden
    driver = webdriver.Firefox()

    # Seite laden
    if year < 2019:
        url = "https://www.marathon.tokyo/" + str(year) + "/result/"        
    else:
        url = "https://www.marathon.tokyo/result/" 
    driver.get(url)
    time.sleep(10)

    # Leeres Dataframe anlegen
    df_results = pd.DataFrame(pd.DataFrame(
    columns=[
        'Jahr'
        ,'Ges'        
        ,'Place_Overall'
        ,'Number_Card'
        ,'Name'
        ,'Age_Group'
        ,'Place_Age_Group'
        ,'Place_Gender'
        ,'Time_net'
        ,'Time_gross'
        ,'5K_Time'
        ,'10K_Time'
        ,'15K_Time'
        ,'20K_Time'
        ,'HK_Time'
        ,'25K_Time'
        ,'30K_Time'
        ,'35K_Time'
        ,'40K_Time'
        ,'FK_Time'
        ])
    ,dtype=str)

    # Ergebnisse auswaehlen und anzeigen    
    for ges in ['M', 'W']:
        if ges == 'M':
            cat_option = "1"
        elif ges == 'W':
            cat_option = "2"
        # Geschlecht auswaehlen
        driver.find_element_by_xpath("//select[@id='category']/option[@value='" + cat_option + "']").click()
        time.sleep(3)
        # Suche-Button
        driver.find_element_by_xpath("//input[@id='btn_submit']").click()

        # Uber die ersten 10 Athleten iterieren (Header uebersprungen)
        for i in range(2,12):
            # Ein Athlet aufrufen        
            xpath_link = "//table[@class='m-item_tbl mb10']/tbody/tr[" + str(i) + "]/td[4]/a[1]"
            driver.find_element_by_xpath(xpath_link).click()
            time.sleep(4)
            xpath_athlet = "//div[@id='result_content']/div[@class='contentsBox']/div[@class='cont3']"
            xpath_athlet_t1 = xpath_athlet + "/table[1]/tbody[1]"
            xpath_athlet_t2 = xpath_athlet + "/table[2]/tbody[1]"
            xpath_athlet_t3 = xpath_athlet + "/table[3]/tbody[1]"
            
            # Auf der naechsten Seite die Daten abziehen

            name = driver.find_element_by_xpath(xpath_athlet_t1 + "/tr[2]/td[3]").text
            age_group = driver.find_element_by_xpath(xpath_athlet_t2 + "/tr[2]/td[2]").text
            place_gender = driver.find_element_by_xpath(xpath_athlet_t2 + "/tr[3]/td[2]").text
            
            result_temp = {
                        'Jahr' : year
                        ,'Ges' : ges
                        ,'Place_Overall' : driver.find_element_by_xpath(xpath_athlet_t1 + "/tr[2]/td[1]").text
                        ,'Number_Card' : driver.find_element_by_xpath(xpath_athlet_t1 + "/tr[2]/td[2]").text
                        ,'Name' : get_decoded_element(name,1)
                        ,'Age_Group' : get_decoded_element(age_group,4) + "-" + get_decoded_element(age_group,3)
                        ,'Place_Age_Group' : get_decoded_element(age_group,2)
                        ,'Place_Gender' : get_decoded_element(place_gender,2)
                        ,'Time_net' : driver.find_element_by_xpath(xpath_athlet_t2 + "/tr[5]/td[1]").text
                        ,'Time_gross' : driver.find_element_by_xpath(xpath_athlet_t2 + "/tr[6]/td[1]").text
                        ,'5K_Time' : driver.find_element_by_xpath(xpath_athlet_t3 + "/tr[2]/td[2]").text
                        ,'10K_Time' : driver.find_element_by_xpath(xpath_athlet_t3 + "/tr[4]/td[2]").text
                        ,'15K_Time' : driver.find_element_by_xpath(xpath_athlet_t3 + "/tr[6]/td[2]").text
                        ,'20K_Time' : driver.find_element_by_xpath(xpath_athlet_t3 + "/tr[8]/td[2]").text
                        ,'HK_Time' : driver.find_element_by_xpath(xpath_athlet_t3 + "/tr[10]/td[2]").text
                        ,'25K_Time' : driver.find_element_by_xpath(xpath_athlet_t3 + "/tr[11]/td[2]").text
                        ,'30K_Time' : driver.find_element_by_xpath(xpath_athlet_t3 + "/tr[13]/td[2]").text
                        ,'35K_Time' : driver.find_element_by_xpath(xpath_athlet_t3 + "/tr[15]/td[2]").text
                        ,'40K_Time' : driver.find_element_by_xpath(xpath_athlet_t3 + "/tr[17]/td[2]").text
                        ,'FK_Time' : driver.find_element_by_xpath(xpath_athlet_t3 + "/tr[19]/td[2]").text
            }
            df_results = df_results.append(result_temp, ignore_index=True)

            # Zurueck zu der Athletenliste
            driver.find_element_by_xpath("//div[@id='result_content']/div[@class='contentsBox']/p[@class='fzM mb10 mt0 linkBack']/a[1]").click()
            time.sleep(3)

    
    # Ergebnisse anzeigen
    # print(df_results.head())

    # Daten in die CSV Exportieren
    file_path = '/home/paul/python_projects/masterthesis/data/Tokyo/'
    file_name = 'daten_wmm_tokyo_' + str(year) + '.csv'
    df_results.to_csv(file_path+file_name, sep=';', index=False)

    # Treiber schliessen
    driver.close()